## BERT as a service

This notebook demonstrates how to build a complete machine learning pipeline using TensorFlow Extended ([TFX](https://www.tensorflow.org/tfx)) to serve a BERT model for text sentiment classification.

Notes:
 - Data: IMDB Movie Reviews (5000 samples) [original source](https://ai.stanford.edu/~amaas/data/sentiment/)
 - Model: BERT base uncased (english) from [HuggingFace](https://huggingface.co/bert-base-uncased)
 - Processor: BERT uncased (english seq_length=128) from [TF HUB](https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3)


You can check references, additional information, and resources at the [GitHub repository](https://github.com/dimitreOliveira/bert-as-a-service_TFX).

## Setup

In [17]:
# Use the latest version of pip.
!pip install --upgrade pip
# Install tfx and kfp Python packages.
!pip install -q --upgrade tfx[kfp]==1.0.0rc1

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-io 0.18.0 requires tensorflow-io-gcs-filesystem==0.18.0, which is not installed.


### Import packages

In [28]:
import os
import sys
import urllib
import tfx
import tensorflow as tf

In [29]:
print(f'TensorFlow version: {tf.__version__}')
print(f'TFX version: {tfx.__version__}')

TensorFlow version: 2.5.0
TFX version: 1.0.0-rc1


[Kubeflow Pipelines](https://www.kubeflow.org/docs/pipelines/overview/pipelines-overview/) environment variables

In [4]:
# Read GCP project id from env.
shell_output=!gcloud config list --format 'value(core.project)' 2>/dev/null
GOOGLE_CLOUD_PROJECT=shell_output[0]
%env GOOGLE_CLOUD_PROJECT={GOOGLE_CLOUD_PROJECT}
print("GCP project ID:" + GOOGLE_CLOUD_PROJECT)

env: GOOGLE_CLOUD_PROJECT=gcp-bert-aas
GCP project ID:gcp-bert-aas


KFP endpoint
"AI Platform > Pipeline > pipeline instance `settings`"
**ENDPOINT should contain only the hostname part of the URL.** For example, if the URL of the KFP dashboard is `https://1e9deb537390ca22-dot-asia-east1.pipelines.googleusercontent.com/#/start`, ENDPOINT value becomes `1e9deb537390ca22-dot-asia-east1.pipelines.googleusercontent.com`.

In [5]:
# This refers to the KFP cluster endpoint
ENDPOINT='4c4887d40ceb4e53-dot-us-central1.pipelines.googleusercontent.com'
if not ENDPOINT:
    from absl import logging
    logging.error('Set your ENDPOINT in this cell.')

Set the image name as `tfx-pipeline` under the current GCP project.

In [6]:
# Docker image name for the pipeline image.
CUSTOM_TFX_IMAGE='gcr.io/' + GOOGLE_CLOUD_PROJECT + '/tfx-pipeline'
print(CUSTOM_TFX_IMAGE)

In [9]:
PIPELINE_NAME="my_pipeline"
PROJECT_DIR=os.path.join(os.path.expanduser("~"), "imported", PIPELINE_NAME)

Change the working directory context in this notebook to the project directory.

In [10]:
%cd {PROJECT_DIR}

/home/jupyter/imported/my_pipeline


## Source files

Here is brief introduction to each of the Python files.
-   `data` - This directory contains the datasets.
-   `models` - This directory contains ML model definitions.
    -   `bert_aas_utils.py` — defines utility functions for the pipeline
-   `pipeline` - This directory contains the definition of the pipeline
    -   `configs.py` — defines common constants for pipeline runners
    -   `pipeline.py` — defines TFX components and a pipeline
-   `local_runner.py`, `kubeflow_runner.py` — define runners for each orchestration engine


### Download data

In [30]:
data_csv_url = 'https://raw.githubusercontent.com/dimitreOliveira/bert-as-a-service_TFX/main/Data/IMDB_5k_dataset.csv'
data_csv_filename = 'IMDB_dataset.csv'

_data_dir = 'data/'
if not os.path.exists(_data_dir):
    os.makedirs(_data_dir)

# Download data
urllib.request.urlretrieve(data_csv_url, f'{_data_dir}{data_csv_filename}')

('data/IMDB_dataset.csv', <http.client.HTTPMessage at 0x7fb759992a10>)

Upload our sample data to GCS bucket so that we can use it in our pipeline later.

In [31]:
!gsutil cp data/IMDB_dataset.csv gs://{GOOGLE_CLOUD_PROJECT}-kubeflowpipelines-default/bert-aas/data/IMDB_dataset.csv

Copying file://data/IMDB_dataset.csv [Content-Type=text/csv]...
/ [1 files][  6.3 MiB/  6.3 MiB]                                                
Operation completed over 1 objects/6.3 MiB.                                      


## Create TFX pipeline

Components in the TFX pipeline will generate outputs for each run as [ML Metadata Artifacts](https://www.tensorflow.org/tfx/guide/mlmd), and they need to be stored somewhere. You can use any storage which the KFP cluster can access, and for this example we will use Google Cloud Storage (GCS). A default GCS bucket should have been created automatically. Its name will be `<your-project-id>-kubeflowpipelines-default`.

Let's create a TFX pipeline using the `tfx pipeline create` command.

>Note: When creating a pipeline for KFP, we need a container image which will be used to run our pipeline. And `skaffold` will build the image for us. Because skaffold pulls base images from the docker hub, it will take 5~10 minutes when we build the image for the first time, but it will take much less time from the second build.

In [24]:
!tfx pipeline create  \
--pipeline-path=kubeflow_runner.py \
--endpoint={ENDPOINT} \
--build-image

While creating a pipeline, `Dockerfile` will be generated to build a Docker image. Don't forget to add it to the source control system (for example, git) along with other source files.

NOTE: `kubeflow` will be automatically selected as an orchestration engine if `airflow` is not installed and `--engine` is not specified.

In [21]:
!tfx run create  \
--pipeline-name={PIPELINE_NAME} \
--endpoint={ENDPOINT}

2021-06-05 14:39:12.884349: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
CLI
Creating a run for pipeline: my_pipeline
Detected Kubeflow.
Use --engine flag if you intend to use a different orchestrator.
/opt/conda/lib/python3.7/site-packages/kfp/_client.py:182: UserWarning: The host 4c4887d40ceb4e53-dot-us-central1.pipelines.googleusercontent.com does not contain the "http" or "https" protocol. Defaults to "https".
  ' Defaults to "https".' % host)
Run created for pipeline: my_pipeline
+===============+======================================+========+===========================+=============================================================================================================================+
| pipeline_name | run_id                               | status | created_at                | link                                                                                                                       

Or, you can also run the pipeline in the KFP Dashboard.  The new execution run will be listed under Experiments in the KFP Dashboard.  Clicking into the experiment will allow you to monitor progress and visualize the artifacts created during the execution run.

However, we recommend visiting the KFP Dashboard. You can access the KFP Dashboard from the Cloud AI Platform Pipelines menu in Google Cloud Console. Once you visit the dashboard, you will be able to find the pipeline, and access a wealth of information about the pipeline.
For example, you can find your runs under the *Experiments* menu, and when you open your execution run under Experiments you can find all your artifacts from the pipeline under *Artifacts* menu.

>Note: If your pipeline run fails, you can see detailed logs for each TFX component in the Experiments tab in the KFP Dashboard.
    
One of the major sources of failure is permission related problems. Please make sure your KFP cluster has permissions to access Google Cloud APIs. This can be configured [when you create a KFP cluster in GCP](https://cloud.google.com/ai-platform/pipelines/docs/setting-up), or see [Troubleshooting document in GCP](https://cloud.google.com/ai-platform/pipelines/docs/troubleshooting).

### In case the TFX pipeline needs to be updated

In [55]:
# Update the pipeline
!tfx pipeline update \
--pipeline-path=kubeflow_runner.py \
--endpoint={ENDPOINT} \
# --build-image

# You can run the pipeline the same way.
!tfx run create \
--pipeline-name {PIPELINE_NAME} \
--endpoint={ENDPOINT}

2021-06-05 20:42:00.534710: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
CLI
Updating pipeline
Detected Kubeflow.
Use --engine flag if you intend to use a different orchestrator.
/opt/conda/lib/python3.7/site-packages/kfp/_client.py:182: UserWarning: The host 4c4887d40ceb4e53-dot-us-central1.pipelines.googleusercontent.com does not contain the "http" or "https" protocol. Defaults to "https".
  ' Defaults to "https".' % host)
INFO:absl:Excluding no splits because exclude_splits is not set.
INFO:absl:Excluding no splits because exclude_splits is not set.
INFO:absl:Excluding no splits because exclude_splits is not set.
[Docker] Step 1/4 : FROM tensorflow/tfx:1.0.0-rc1[Docker] 
[Docker] The push refers to repository [gcr.io/gcp-bert-aas/my_pipeline]
[Docker] Preparing
[Docker] Preparing
[Docker] Preparing
[Docker] Preparing
[Docker] Preparing
[Docker] Preparing
[Docker] Preparing
[Docker] Preparing
[Docker] Preparing


### Check pipeline outputs

Visit the KFP dashboard to find pipeline outputs in the page for your pipeline run. Click the *Experiments* tab on the left, and *All runs* in the Experiments page. You should be able to find the latest run under the name of your pipeline.

**NOTE:** If we changed anything in the model code, we have to rebuild the
container image, too. We can trigger rebuild using `--build-image` flag in the
`pipeline update` command.

**NOTE:** You might have noticed that every time we create a pipeline run, every component runs again and again even though the input and the parameters were not changed.
It is waste of time and resources, and you can skip those executions with pipeline caching. You can enable caching by specifying `enable_cache=True` for the `Pipeline` object in `pipeline.py`.
